In [2]:
import PySimpleGUI as sg
import oct_tools as ot
from oct_tools import Mzi, Bpd
from scipy.signal import savgol_filter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import figure

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# plt.style.use('dark_background')
plt.style.use('bmh')
# Set the background color to dark
plt.rcParams['axes.facecolor'] = '#000000'
plt.rcParams['figure.facecolor'] = '#1a1a1a'
# # Set legend text color to white
# plt.rcParams['legend.facecolor'] = 'none'  # Set legend background color to transparent
# plt.rcParams['legend.labelcolor'] = 'white'  # Set legend label color to white


In [3]:
## VARIABLE SETUP
EXITALL = 'EXITALL'

INPUT_POWER_INPUT = 'INPUT_POWER_INPUT'
SWEEP_FREQUENCY_INPUT = 'SWEEP_FREQUENCY_INPUT'
SWEEP_PARAM_INPUT_SIZE = 5
SWEEP_PARAM_INPUT_FONT_SIZE = 20


MZI_LENGTH_SLIDER = 'MZI_LENGTH_SLIDER'
MZI_LENGTH_SLIDER_DEFAULT = 4.0 # mm
MZI_LENGTH_SLIDER_RESOLUTION = 0.1
MZI_LENGTH_SLIDER_RANGE = (0.1, 100) # mm
MZI_LENGTH_SLIDER_INPUT = 'MZI_LENGTH_SLIDER_INPUT'
MZI_LENGTH_INPUT_SIZE = 6
MZI_FSR_TEXT = '-MZI_FSR_TEXT-'

MEAS_DISTANCE_SLIDER = 'MEAS_DISTANCE_SLIDER'
MEAS_DISTANCE_SLIDER_DEFAULT = 2.0 # mm
MEAS_DISTANCE_SLIDER_RESOLUTION = MZI_LENGTH_SLIDER_RESOLUTION / 2
MEAS_DISTANCE_SLIDER_RANGE = (0.1, MZI_LENGTH_SLIDER_RANGE[1] / 2) # mm
MEAS_DISTANCE_SLIDER_INPUT = 'MEAS_DISTANCE_SLIDER_INPUT'
MEAS_DISTANCE_INPUT_SIZE = 6
MEAS_FSR_TEXT = '-MEAS_FSR_TEXT-'

INPUT_COUPLER_H = 'INPUT_COUPLER_H'
INPUT_COUPLER_L = 'INPUT_COUPLER_L'
OUTPUT_COUPLER_H = 'OUTPUT_COUPLER_H'
OUTPUT_COUPLER_L = 'OUTPUT_COUPLER_L'

CROP_THRESHOLD_INPUT = '-CROP_THRESHOLD_INPUT'


BPD_BANDWIDTH_INPUT = '-BPD_BANDWIDTH_INPUT'
BPD_INPUT_SIZE = 7

BPD_GAIN_SLIDER = '-BPD_GAIN_SLIDER'
BPD_GAIN_SLIDER_DEFAULT = '5000'
BPD_GAIN_SLIDER_RESOLUTION = 100
BPD_GAIN_SLIDER_RANGE = (100, 20_000)
BPD_GAIN_SLIDER_INPUT = '-BPD_GAIN_SLIDER_INPUT'
BPD_GAIN_INPUT_SIZE = 7

BPD_NOISE_SLIDER = '-BPD_NOISE_SLIDER'
BPD_NOISE_SLIDER_DEFAULT = 5e-12
BPD_NOISE_SLIDER_RESOLUTION = 1e-12
BPD_NOISE_SLIDER_RANGE = (0, 30e-12)
BPD_NOISE_SLIDER_INPUT = '-BPD_NOISE_SLIDER_INPUT'
BPD_NOISE_INPUT_SIZE = 7


SELECT_FBG_INPUT = 'SELECT_FBG_INPUT'
SELECT_FBG_BUTTON = 'SELECT_FBG_BUTTON'

SELECT_POWER_SHAPE_INPUT = 'SELECT_POWER_SHAPE_INPUT'
SELECT_POWER_SHAPE_BUTTON = 'SELECT_POWER_SHAPE_BUTTON'

RUN_BUTTON = 'RUN_BUTTON'
RUN_BUTTON_SIZE = (25, 4)
RUN_BUTTON_FONT_SIZE = '72'

FRINGE_CANVAS = 'FRINGE_CANVAS'
FFT_CANVAS = 'FFT_CANVAS'
PLOT_STYLE = 'PLOT_STYLE'
PLOT_SELECT_RADIO_GROUP = 'PLOT_SELECT_RADIO_GROUP'
PLOT_SELECT_RADIO_RAW_FBG = 'PLOT_SELECT_RADIO_RAW_FBG'
PLOT_SELECT_RADIO_FBG_POWER_SHAPED = 'PLOT_SELECT_RADIO_FBG_POWER_SHAPED'
PLOT_SELECT_RADIO_DETECTION_OUTPUT = 'PLOT_SELECT_RADIO_DETECTED_OUTPUT'

BUTTON_SIZE = (10, 2)
SELECTED_BUTTON_COLOR = sg.PURPLES[0]

BUTTON_SIZE = (10, 2)


sg.theme('Topanga');

In [4]:
# ------------------------------- WINDOWS -------------------------------
def make_main_window() -> sg.Window:
    ''' Makes main window
    '''
    sweep_parameter_text_column_layout = [[sg.Text('Input Power (mW)')],
                                          [sg.Text('Sweep Frequency (kHz)')],
                                          ]
    sweep_parameter_input_column_layout = [[sg.Input('2.5', key=INPUT_POWER_INPUT, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                                           [sg.Input('50', key=SWEEP_FREQUENCY_INPUT, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                                           ]
    sweep_parameters_frame_layout = [[sg.Column(sweep_parameter_text_column_layout), sg.Column(sweep_parameter_input_column_layout)],
                                    ]
    
    
    mzi_text_column_layout = [[sg.Text('Input Coupler Split (%)')],
                              [sg.Text('Output Coupler Split (%)')],
                              ]
    mzi_input_column_layout = [[sg.Input('50', key=INPUT_COUPLER_H, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE), sg.Input('50', key=INPUT_COUPLER_L, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                               [sg.Input('50', key=OUTPUT_COUPLER_H, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE), sg.Input('50', key=OUTPUT_COUPLER_L, size=SWEEP_PARAM_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                               ]
    mzi_frame_layout = [[sg.Text('Length in Air (mm)')],
                        [sg.Slider(MZI_LENGTH_SLIDER_RANGE, key=MZI_LENGTH_SLIDER, default_value=MZI_LENGTH_SLIDER_DEFAULT, enable_events=True, orientation='h', resolution=MZI_LENGTH_SLIDER_RESOLUTION, disable_number_display=True),
                         sg.Input(MZI_LENGTH_SLIDER_DEFAULT, key=MZI_LENGTH_SLIDER_INPUT, size=MZI_LENGTH_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE, enable_events=True)],
                         [sg.Text('FSR:'), sg.Text('', key=MZI_FSR_TEXT), sg.Text('pm')],
                        [sg.Column(mzi_text_column_layout), sg.Column(mzi_input_column_layout)],
                        [sg.Text('Measurement Distance (mm)')],
                        [sg.Slider(MEAS_DISTANCE_SLIDER_RANGE, key=MEAS_DISTANCE_SLIDER, default_value=MEAS_DISTANCE_SLIDER_DEFAULT, enable_events=True, orientation='h', resolution=MEAS_DISTANCE_SLIDER_RESOLUTION, disable_number_display=True),
                         sg.Input(MEAS_DISTANCE_SLIDER_DEFAULT, key=MEAS_DISTANCE_SLIDER_INPUT, size=MEAS_DISTANCE_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE, enable_events=True)],
                        [sg.Text('FSR:'), sg.Text('', key=MEAS_FSR_TEXT), sg.Text('pm')],
                        ]

    bpd_text_column_layout = [[sg.Text('Bandwidth (Hz)')],
                            ]
    bpd_input_column_layout = [[sg.Input('500e6', key=BPD_BANDWIDTH_INPUT, size=BPD_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)],
                            ]
    bpd_frame_layout = [[sg.Column(bpd_text_column_layout), sg.Column(bpd_input_column_layout)],
                        [sg.Text('Gain (V/A)')],
                        [sg.Slider(BPD_GAIN_SLIDER_RANGE, key=BPD_GAIN_SLIDER, default_value=BPD_GAIN_SLIDER_DEFAULT, enable_events=True, orientation='h', resolution=BPD_GAIN_SLIDER_RESOLUTION, disable_number_display=True),
                         sg.Input(BPD_GAIN_SLIDER_DEFAULT, key=BPD_GAIN_SLIDER_INPUT, size=BPD_GAIN_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE, enable_events=True)],
                        [sg.Text('Noise Density (A/rtHz)')],
                        [sg.Slider(BPD_NOISE_SLIDER_RANGE, key=BPD_NOISE_SLIDER, default_value=BPD_NOISE_SLIDER_DEFAULT, enable_events=True, orientation='h', resolution=BPD_NOISE_SLIDER_RESOLUTION, disable_number_display=True),
                         sg.Input(BPD_NOISE_SLIDER_DEFAULT, key=BPD_NOISE_SLIDER_INPUT, size=BPD_NOISE_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE, enable_events=True)],
                        ]

    
    misc_frame_layout = [[sg.Text('Crop Threshold (%)'), sg.Input('5', key=CROP_THRESHOLD_INPUT, size=MEAS_DISTANCE_INPUT_SIZE, font=SWEEP_PARAM_INPUT_FONT_SIZE)]]

    
    entry_column_layout = [[sg.Frame('Sweep Parameters', sweep_parameters_frame_layout)], 
                        [sg.Frame('Reference MZI Parameters', mzi_frame_layout)],
                        [sg.Frame('Misc.', misc_frame_layout)],
                        [sg.Frame('Balanced Detector', bpd_frame_layout)],
                        [sg.Text()],
                        [sg.Push(), sg.Button('Run', key=RUN_BUTTON, size=RUN_BUTTON_SIZE, font=RUN_BUTTON_FONT_SIZE), sg.Push()]]


    frame_layout_plotting_area = [[sg.Push(), sg.Radio('Raw FBG', PLOT_SELECT_RADIO_GROUP, key=PLOT_SELECT_RADIO_RAW_FBG, enable_events=True),
                                   sg.Radio('Power Shaped FBG', PLOT_SELECT_RADIO_GROUP, key=PLOT_SELECT_RADIO_FBG_POWER_SHAPED, enable_events=True),
                                   sg.Radio('Detection Output', PLOT_SELECT_RADIO_GROUP, key=PLOT_SELECT_RADIO_DETECTION_OUTPUT, enable_events=True), sg.Push()],
                                  [sg.Canvas(key=FRINGE_CANVAS, size=(640, 480), background_color='black')],
                                  [sg.Canvas(key=FFT_CANVAS, size=(640, 480), background_color='black')],
                                  [sg.Push(), sg.Combo(plt.style.available, size=(15, 10), key=PLOT_STYLE, enable_events=True)]]


    layout = [[sg.Push(), sg.Text('FBG Detection Analyzer', font=48), sg.Push()],
              [sg.Column(entry_column_layout, vertical_alignment='top'), sg.Push(), sg.Frame('Plotting', frame_layout_plotting_area)],
              [sg.VPush()],
              [sg.Button('Exit', key=EXITALL, size=BUTTON_SIZE, font=24)]]

    return sg.Window('FBG Detection Analyzer', layout, size=(1920, 1080), resizable=True, finalize=True)

def close_all_windows(windows : list[sg.Window]) -> None:
    for window in windows:
        if window is not None:
            window.close()

# ------------------------------- PSG HELPERS -------------------------------
def draw_figure(canvas, figure) -> FigureCanvasTkAgg:
    figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
    figure_canvas_agg.draw()
    figure_canvas_agg.get_tk_widget().pack(side='top', fill='both', expand=1)
    return figure_canvas_agg


# ------------------------------- PLOTTING -------------------------------
def make_figures(window: sg.Window):
    fig0 = figure.Figure(figsize=(100, 5), dpi=100)
    ax0 = fig0.add_subplot(111)
    fig1 = figure.Figure(figsize=(100, 5), dpi=100)
    ax1 = fig1.add_subplot(111)

    fringe_fig_canvas_agg = draw_figure(window[FRINGE_CANVAS].TKCanvas, fig0)
    fft_fig_canvas_agg = draw_figure(window[FFT_CANVAS].TKCanvas, fig1)

    return fig0, ax0, fig1, ax1, fringe_fig_canvas_agg, fft_fig_canvas_agg

def plot_fringe(t, reference_voltage, meas_voltage, fig_canvas_agg, ax) -> None:
    ax.cla()
    ax.plot(t, reference_voltage, label='Reference Fringe')
    ax.plot(t, meas_voltage, label='Measurement Fringe')
    ax.set_ylabel('Detection Voltage (V)')
    ax.set_xlabel('Time (s)')
    ax.legend()
    fig_canvas_agg.draw()

def plot_fft(freq, reference_fft, meas_fft, fig_canvas_agg, ax) -> None:
    ax.cla()
    ax.plot(freq, reference_fft, label='Reference', linewidth=0.5)
    ax.plot(freq, meas_fft, label='Measurement', linewidth=0.5)
    ax.set_ylabel('Magnitude')
    ax.set_xlabel('Frequency (Hz)')
    ax.legend()
    fig_canvas_agg.draw()

# ------------------------------- DATA PROCESSING -------------------------------
def calculate_fft(T, signal):
    '''T is sample spacing of signal
    Returns frequency, fft complex data
    '''
    n = len(signal)
    freq = np.fft.fftfreq(n, T)[: n // 2]  # Frequency values
    fft_result = np.fft.fft(signal)[: n // 2] / n  # FFT result (normalized)

    return freq, fft_result

def get_lin_data_arrays(lin_data, average_power_in):
    t = lin_data['Time'].to_numpy()
    wl = lin_data['Wavelength'].to_numpy() * 1e-9
    boa_out = np.abs(lin_data['PowerEnvelope'].to_numpy())
    # boa_out = boa_out / np.max(boa_out)
    boa_out = boa_out * average_power_in / np.mean(boa_out) # normalize to average power
    
    return t, wl, boa_out


In [5]:
windows = [main_window, new_window,] = [make_main_window(), None,]

fig0, ax0, fig1, ax1, fringe_fig_canvas_agg, fft_fig_canvas_agg = make_figures(main_window)


_, values = main_window.read(timeout=0)
lin_data = pd.read_csv(r'B4363_V4367_Final_LinData.csv')
ref_mzi = Mzi(float(values[MZI_LENGTH_SLIDER_INPUT]) * 1e-3, float(values[INPUT_COUPLER_H]) / 100, float(values[OUTPUT_COUPLER_H]) / 100)
meas_mzi = Mzi(float(values[MEAS_DISTANCE_SLIDER_INPUT]) * 1e-3, float(values[INPUT_COUPLER_H]) / 100, float(values[OUTPUT_COUPLER_H]) / 100)
bpd = Bpd(i_noise = float(values[BPD_NOISE_SLIDER_INPUT]))

while True:
    window, event, values = sg.read_all_windows()
    print(event)
    
    if window == main_window and event in (sg.WIN_CLOSED, EXITALL):
        break
    
    if window == main_window:
        if event == RUN_BUTTON:
            # Update
            ref_mzi = Mzi(float(values[MZI_LENGTH_SLIDER_INPUT]) * 1e-3, float(values[INPUT_COUPLER_H]) / 100, float(values[OUTPUT_COUPLER_H]) / 100)
            meas_mzi = Mzi(float(values[MEAS_DISTANCE_SLIDER_INPUT]) * 2 * 1e-3, float(values[INPUT_COUPLER_H]) / 100, float(values[OUTPUT_COUPLER_H]) / 100)
            bpd = Bpd(gain = float(values[BPD_GAIN_SLIDER_INPUT]), bandwidth = float(values[BPD_BANDWIDTH_INPUT]), i_noise = float(values[BPD_NOISE_SLIDER_INPUT]))

            # Calculate all
            t, wl, power_out = get_lin_data_arrays(lin_data, float(values[INPUT_POWER_INPUT]))
            # power_out = savgol_filter(power_out, 51, 1) # TODO: handle filtering for different inputs
            t, ref_pout1, ref_pout2 = ref_mzi.calculate_from_time(t, wl, power_out)
            t, meas_pout1, meas_pout2 = meas_mzi.calculate_from_time(t, wl, power_out)
            
            v_ref = bpd.detect(ref_pout1, ref_pout2) # TODO: don't need this if below in averaging
            v_meas = bpd.detect(meas_pout1, meas_pout2) # TODO: don't need this if below in averaging

            # Window
            idx = ot.get_crop_indices(power_out, min(float(values[CROP_THRESHOLD_INPUT]) / 100, 0.5))
            # Calculate FFTs
            ref_signal = ot.apply_hanning(v_ref[idx[0] : idx[1]])

            # TODO: fft averages (efficiently)
            n_avgs = 16
            fft_freq = calculate_fft(t[1] - t[0], ref_signal)[0]
            
            ref_fft = []
            meas_fft = []
            for i in range(n_avgs):
                v_ref = bpd.detect(ref_pout1, ref_pout2)
                v_meas = bpd.detect(meas_pout1, meas_pout2)
                # Window
                idx = ot.get_crop_indices(power_out, min(float(values[CROP_THRESHOLD_INPUT]) / 100, 0.5))
                # Calculate FFTs
                ref_signal = ot.apply_hanning(v_ref[idx[0] : idx[1]])
                meas_signal = ot.apply_hanning(v_meas[idx[0] : idx[1]])
                ref_fft.append(np.abs(calculate_fft(t[1], ref_signal)[1]))
                meas_fft.append(np.abs(calculate_fft(t[1], ot.resample(meas_signal, ref_signal))[1]))

            ref_fft = np.mean(ref_fft, axis=0)
            meas_fft = np.mean(meas_fft, axis=0)

            # Update display
            window[MZI_FSR_TEXT].update(str(round(ot.calculate_fsr_wavelength(np.mean(wl), ref_mzi.fsr) * 1e12, 1)))
            window[MEAS_FSR_TEXT].update(str(round(ot.calculate_fsr_wavelength(np.mean(wl), meas_mzi.fsr) * 1e12, 1)))
            # Plot
            plot_fringe(lin_data['Time'].to_numpy(), v_ref, v_meas, fringe_fig_canvas_agg, ax0)
            plot_fft(fft_freq, 10 * np.log10((np.abs(ref_fft))), 10 * np.log10(np.abs(meas_fft)), fft_fig_canvas_agg, ax1)
            
        # MZI Parameters
        elif event in (MZI_LENGTH_SLIDER, MEAS_DISTANCE_SLIDER, BPD_GAIN_SLIDER, BPD_NOISE_SLIDER): # update inputs to share the slider value
            main_window[event + '_INPUT'].update(values[event])
            window.write_event_value(RUN_BUTTON, '')
        elif event in (MZI_LENGTH_SLIDER_INPUT, MEAS_DISTANCE_SLIDER_INPUT, BPD_GAIN_SLIDER_INPUT, BPD_NOISE_SLIDER_INPUT):
            if values[event] == '':
                continue
            main_window[event.strip('_INPUT')].update(values[event])
            window.write_event_value(RUN_BUTTON, '')

        elif event == PLOT_STYLE:
            plt.style.use(values[PLOT_STYLE])
            plt.close('all')
            # plt.close(fig1)
            window.write_event_value(RUN_BUTTON, '')


close_all_windows(windows)

-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
-BPD_NOISE_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MZI_LENGTH_SLIDER
RUN_BUTTON
MEAS_DISTANCE_